In [ ]:
!pip install -q transformers

     |████████████████████████████████| 3.4 MB 8.5 MB/s 
     |████████████████████████████████| 67 kB 7.3 MB/s 
     |████████████████████████████████| 895 kB 70.7 MB/s 
     |████████████████████████████████| 596 kB 76.5 MB/s 
     |████████████████████████████████| 3.3 MB 50.7 MB/s 


## Change the following path accordingly

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
import os 
os.chdir("/content/drive/MyDrive/DataScienceLab/NER/templateNER")
os.listdir()

## IMPORTANT: Please update the LABELS in according to your dataset

In [ ]:
LABELS=["Adjective","API","Core","GUI","Hardware","Language","Platform","Standard","User","Verb","O"]
entity_dict={i:e for i, e in enumerate(LABELS)}
template_list=[" is a %s entity"%(e) for e in LABELS[:-1]]
template_list.append( " is not a named entity .")

In [ ]:
from utils_metrics import get_entities_bio, f1_score, classification_report
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig
import torch
import time
import math

class InputExample():
    def __init__(self, words, labels):
        self.words = words
        self.labels = labels

def template_entity(words, input_TXT, start):
    LEN=len(template_list)
    words_length = len(words)
    words_length_list = [len(i) for i in words]
    input_TXT = [input_TXT]*(LEN*words_length)

    input_ids = tokenizer(input_TXT, return_tensors='pt')['input_ids']
    model.to(device)
    

    temp_list = []
    for i in range(words_length):
        for j in range(len(template_list)):
            temp_list.append(words[i]+template_list[j])

    output_ids = tokenizer(temp_list, return_tensors='pt', padding=True, truncation=True)['input_ids']
    output_ids[:, 0] = 2
    output_length_list = [0]*LEN*words_length


    for i in range(len(temp_list)//LEN):
        base_length = ((tokenizer(temp_list[i * LEN], return_tensors='pt', padding=True, truncation=True)['input_ids']).shape)[1] - 4
        output_length_list[i*LEN:i*LEN+ LEN] = [base_length]*LEN
        output_length_list[i*LEN+4] += 1

    score = [1]*LEN*words_length
    with torch.no_grad():
        output = model(input_ids=input_ids.to(device), decoder_input_ids=output_ids[:, :output_ids.shape[1] - 2].to(device))[0]
        for i in range(output_ids.shape[1] - 3):
            # print(input_ids.shape)
            logits = output[:, i, :]
            logits = logits.softmax(dim=1)
            # values, predictions = logits.topk(1,dim = 1)
            logits = logits.to('cpu').numpy()
            # print(output_ids[:, i+1].item())
            for j in range(0, LEN*words_length):
                if i < output_length_list[j]:
                    score[j] = score[j] * logits[j][int(output_ids[j][i + 1])]

    end = start+(score.index(max(score))//LEN)
        # score_list.append(score)
    return [start, end, entity_dict[(score.index(max(score))%LEN)], max(score)] #[start_index,end_index,label,score]



def prediction(input_TXT):
    input_TXT_list = input_TXT.split(' ')
    entity_list = []
    for i in range(len(input_TXT_list)):
        words = []
        for j in range(1, min(9, len(input_TXT_list) - i + 1)):
            word = (' ').join(input_TXT_list[i:i+j])
            words.append(word)

        entity = template_entity(words, input_TXT, i) #[start_index,end_index,label,score]
        if entity[1] >= len(input_TXT_list):
            entity[1] = len(input_TXT_list)-1
        if entity[2] != 'O':
            entity_list.append(entity)
        #print(words)
    i = 0
    if len(entity_list) > 1:
        while i < len(entity_list):
            j = i+1
            while j < len(entity_list):
                if (entity_list[i][1] < entity_list[j][0]) or (entity_list[i][0] > entity_list[j][1]):
                    j += 1
                else:
                    if entity_list[i][3] < entity_list[j][3]:
                        entity_list[i], entity_list[j] = entity_list[j], entity_list[i]
                        entity_list.pop(j)
                    else:
                        entity_list.pop(j)
            i += 1
    label_list = ['O'] * len(input_TXT_list)
    #print("entity list ",entity_list)

    for entity in entity_list:
        label_list[entity[0]:entity[1]+1] = ["I-"+entity[2]]*(entity[1]-entity[0]+1)
        label_list[entity[0]] = "B-"+entity[2]
    return label_list

def cal_time(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
# outputs/best_model is suppoed to be trained and saved with Template NER Training.ipynb 
model = BartForConditionalGeneration.from_pretrained('./outputs/best_model')
# For zero-shot learning
#model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
model.eval()
model.config.use_cache = False

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

## Inference

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
input="As a user I should be able to use the attribute type User in my queries."
output=prediction(input)
print(list(zip(input.split(),output)))

[('As', 'O'), ('a', 'O'), ('user', 'B-User'), ('I', 'O'), ('should', 'O'), ('be', 'O'), ('able', 'B-Verb'), ('to', 'O'), ('use', 'B-Verb'), ('the', 'O'), ('attribute', 'B-Core'), ('type', 'I-Core'), ('User', 'B-User'), ('in', 'O'), ('my', 'O'), ('queries.', 'B-Core')]


In [ ]:
!wc -l "data/val1.json" 

1253 data/val1.json


In [ ]:
preds=[]
y_true=[]
for i,line in enumerate(open("data/val1.json")):
  print(i)
  e= eval(line)
  sentence=" ".join(e["tokens"])
  labels= e["labels"]
  output= prediction(sentence)
  if len(output)== len(labels):
    preds+= output
    y_true+= e["labels"]
  else:
    print("miss-matched:")
    print(output, e["labels"])

In [ ]:
from sklearn.metrics import classification_report


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
print(classification_report(y_true, preds))

              precision    recall  f1-score   support

       B-API       1.00      0.50      0.67         2
 B-Adjective       1.00      0.43      0.61        23
      B-Core       0.72      0.85      0.78        94
       B-GUI       0.65      0.72      0.68        54
  B-Hardware       0.00      0.00      0.00         6
  B-Language       1.00      0.33      0.50         3
  B-Platform       0.67      0.73      0.70        11
  B-Standard       0.75      0.35      0.48        17
      B-User       0.81      0.81      0.81        81
      B-Verb       0.87      0.90      0.89       226
       I-API       1.00      1.00      1.00         1
      I-Core       0.17      0.17      0.17         6
       I-GUI       0.75      0.25      0.38        12
  I-Platform       0.00      0.00      0.00         3
  I-Standard       0.00      0.00      0.00         2
      I-User       0.92      0.49      0.64        45
           O       0.91      0.94      0.93      1053

    accuracy              

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(confusion_matrix(y_true, preds))

[[  1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   1]
 [  0  10   0   0   0   0   0   0   0   3   0   0   0   0   0   0  10]
 [  0   0  80   0   0   0   0   0   0   0   0   0   0   0   0   0  14]
 [  0   0   2  39   0   0   0   0   2   1   0   0   0   0   0   0  10]
 [  0   0   1   1   0   0   0   0   0   0   0   0   0   0   0   0   4]
 [  0   0   1   0   0   1   0   0   0   0   0   0   0   0   0   0   1]
 [  0   0   0   2   0   0   8   0   0   1   0   0   0   0   0   0   0]
 [  0   0   4   0   0   0   0   6   0   0   0   0   0   0   0   0   7]
 [  0   0   2   0   0   0   0   0  66   0   0   0   0   0   0   0  13]
 [  0   0   1   1   0   0   0   0   0 204   0   0   0   0   0   0  20]
 [  0   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0]
 [  0   0   2   2   0   0   0   0   0   0   0   1   0   0   0   0   1]
 [  0   0   1   2   0   0   0   0   0   2   0   1   3   0   0   1   2]
 [  0   0   1   1   0   0   0   0   0   0   0   0   1   0   0   0   0]
 [  0 

In [ ]:
res=[a==b for  a,b in zip(y_true,preds)]

In [ ]:
sum(res) /len(res)

0.8673390224980605